![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# 🧠 Section 3: Memory Architecture - From Stateless RAG to Stateful Conversations

**⏱️ Estimated Time:** 45-60 minutes

## 🎯 Learning Objectives

By the end of this notebook, you will:

1. **Understand** why memory is essential for context engineering
2. **Implement** working memory for conversation continuity
3. **Use** long-term memory for persistent user knowledge
4. **Integrate** memory with your Section 2 RAG system
5. **Build** a complete memory-enhanced course advisor

---

## 🔗 Recap

### **Section 1: The Four Context Types**

Recall the four context types from Section 1:

1. **System Context** (Static) - Role, instructions, guidelines
2. **User Context** (Dynamic, User-Specific) - Profile, preferences, goals
3. **Conversation Context** (Dynamic, Session-Specific) - **← Memory enables this!**
4. **Retrieved Context** (Dynamic, Query-Specific) - RAG results

### **Section 2: Stateless RAG**

Your Section 2 RAG system was **stateless**:

```python
async def rag_query(query, student_profile):
    # 1. Search courses (Retrieved Context)
    courses = await course_manager.search_courses(query)

    # 2. Assemble context (System + User + Retrieved)
    context = assemble_context(system_prompt, student_profile, courses)

    # 3. Generate response
    response = llm.invoke(context)

    # ❌ No conversation history stored
    # ❌ Each query is independent
    # ❌ Can't reference previous messages
```

**The Problem:** Every query starts from scratch. No conversation continuity.

---

## 🚨 Why Agents Need Memory: The Grounding Problem

Before diving into implementation, let's understand the fundamental problem that memory solves.

**Grounding** means understanding what users are referring to. Natural conversation is full of references:

### **Without Memory:**

```
User: "Tell me about CS401"
Agent: "CS401 is Machine Learning. It covers supervised learning..."

User: "What are its prerequisites?"
Agent: ❌ "What does 'it' refer to? Please specify which course."

User: "The course we just discussed!"
Agent: ❌ "I don't have access to previous messages. Which course?"
```

**This is a terrible user experience.**

### Types of References That Need Grounding

**Pronouns:**
- "it", "that course", "those", "this one"
- "he", "she", "they" (referring to people)

**Descriptions:**
- "the easy one", "the online course"
- "my advisor", "that professor"

**Implicit context:**
- "Can I take it?" → Take what?
- "When does it start?" → What starts?

**Temporal references:**
- "you mentioned", "earlier", "last time"

### **With Memory:**

```
User: "Tell me about CS401"
Agent: "CS401 is Machine Learning. It covers..."
[Stores: User asked about CS401]

User: "What are its prerequisites?"
Agent: [Checks memory: "its" = CS401]
Agent: ✅ "CS401 requires CS201 and MATH301"

User: "Can I take it?"
Agent: [Checks memory: "it" = CS401, checks student transcript]
Agent: ✅ "You've completed CS201 but still need MATH301"
```

**Now the conversation flows naturally!**

---

## 🧠 Two Types of Memory

### **1. Working Memory (Session-Scoped)**

 - **What:** Conversation messages from the current session
 - **Purpose:** Reference resolution, conversation continuity
 - **Lifetime:** Session duration (24 hours TTL by default)

**Example:**
```
Session: session_123
Messages:
  1. User: "Tell me about CS401"
  2. Agent: "CS401 is Machine Learning..."
  3. User: "What are its prerequisites?"
  4. Agent: "CS401 requires CS201 and MATH301"
```

### **2. Long-term Memory (Cross-Session)**

 - **What:** Persistent facts, preferences, goals
 - **Purpose:** Personalization across sessions and applications
 - **Lifetime:** Permanent (until explicitly deleted)

**Example:**
```
User: student_sarah
Memories:
  - "Prefers online courses over in-person"
  - "Major: Computer Science, focus on AI/ML"
  - "Goal: Graduate Spring 2026"
  - "Completed: CS101, CS201, MATH301"
```

### **Comparison: Working vs. Long-term Memory**

| Working Memory | Long-term Memory |
|----------------|------------------|
| **Session-scoped** | **User-scoped** |
| Current conversation | Important facts |
| TTL-based (expires) | Persistent |
| Full message history | Extracted knowledge |
| Loaded/saved each turn | Searched when needed |

---

## 📦 Setup and Environment

Let's set up our environment with the necessary dependencies and connections. We'll build on Section 2's RAG foundation and add memory capabilities.

### ⚠️ Prerequisites

**Before running this notebook, make sure you have:**

1. **Docker Desktop running** - Required for Redis and Agent Memory Server

2. **Environment variables** - Create a `.env` file in the `reference-agent` directory:
   ```bash
   # Copy the example file
   cd ../../reference-agent
   cp .env.example .env

   # Edit .env and add your OpenAI API key
   # OPENAI_API_KEY=your_actual_openai_api_key_here
   ```

3. **Run the setup script** - This will automatically start Redis and Agent Memory Server:
   ```bash
   cd ../../reference-agent
   python setup_agent_memory_server.py
   ```

**Note:** The setup script will:
- ✅ Check if Docker is running
- ✅ Start Redis if not running (port 6379)
- ✅ Start Agent Memory Server if not running (port 8088)
- ✅ Verify Redis connection is working
- ✅ Handle any configuration issues automatically

If the Memory Server is not available, the notebook will skip memory-related demos but will still run.


---


### Automated Setup Check

Let's run the setup script to ensure all services are running properly.


In [34]:
# Run the setup script to ensure Redis and Agent Memory Server are running
import subprocess
import sys
from pathlib import Path

# Path to setup script
setup_script = Path("../../reference-agent/setup_agent_memory_server.py")

if setup_script.exists():
    print("Running automated setup check...\n")
    result = subprocess.run(
        [sys.executable, str(setup_script)],
        capture_output=True,
        text=True
    )
    print(result.stdout)
    if result.returncode != 0:
        print("⚠️  Setup check failed. Please review the output above.")
        print(result.stderr)
    else:
        print("\n✅ All services are ready!")
else:
    print("⚠️  Setup script not found. Please ensure services are running manually.")


Running automated setup check...


🔧 Agent Memory Server Setup
📊 Checking Redis...
✅ Redis is running
📊 Checking Agent Memory Server...
🔍 Agent Memory Server container exists. Checking health...
✅ Agent Memory Server is running and healthy
✅ No Redis connection issues detected

✅ Setup Complete!
📊 Services Status:
   • Redis: Running on port 6379
   • Agent Memory Server: Running on port 8088

🎯 You can now run the notebooks!


✅ All services are ready!


---


### Install Dependencies

If you haven't already installed the reference-agent package, uncomment and run the following:


In [35]:
# Uncomment to install reference-agent package
# %pip install -q -e ../../reference-agent

# Uncomment to install agent-memory-client
# %pip install -q agent-memory-client


### Load Environment Variables

We'll load environment variables from the `.env` file in the `reference-agent` directory.

**Required variables:**
- `OPENAI_API_KEY` - Your OpenAI API key
- `REDIS_URL` - Redis connection URL (default: redis://localhost:6379)
- `AGENT_MEMORY_URL` - Agent Memory Server URL (default: http://localhost:8088)

If you haven't created the `.env` file yet, copy `.env.example` and add your OpenAI API key.


In [36]:
import os
from pathlib import Path
from dotenv import load_dotenv

# Load environment variables from reference-agent directory
env_path = Path("../../reference-agent/.env")
load_dotenv(dotenv_path=env_path)

# Verify required environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379")
AGENT_MEMORY_URL = os.getenv("AGENT_MEMORY_URL", "http://localhost:8088")

if not OPENAI_API_KEY:
    print(f"""❌ OPENAI_API_KEY not found!

    Please create a .env file at: {env_path.absolute()}

    With the following content:
    OPENAI_API_KEY=your_openai_api_key
    REDIS_URL=redis://localhost:6379
    AGENT_MEMORY_URL=http://localhost:8088
    """)
else:
    print("✅ Environment variables loaded")
    print(f"   REDIS_URL: {REDIS_URL}")
    print(f"   AGENT_MEMORY_URL: {AGENT_MEMORY_URL}")


✅ Environment variables loaded
   REDIS_URL: redis://localhost:6379
   AGENT_MEMORY_URL: http://localhost:8088


### Import Core Libraries

We'll import standard Python libraries and async support for our memory operations.


In [37]:
import asyncio
from typing import List, Dict, Any, Optional
from datetime import datetime

print("✅ Core libraries imported")


✅ Core libraries imported


### Import Section 2 Components

We're building on Section 2's RAG foundation, so we'll reuse the same components:
- `redis_config` - Redis connection and configuration
- `CourseManager` - Course search and management
- `StudentProfile` and other models - Data structures


In [38]:
# Import Section 2 components from reference-agent
from redis_context_course.redis_config import redis_config
from redis_context_course.course_manager import CourseManager
from redis_context_course.models import (
    Course, StudentProfile, DifficultyLevel,
    CourseFormat, Semester
)

print("✅ Section 2 components imported")
print(f"   CourseManager: Available")
print(f"   Redis Config: Available")
print(f"   Models: Course, StudentProfile, etc.")


✅ Section 2 components imported
   CourseManager: Available
   Redis Config: Available
   Models: Course, StudentProfile, etc.


### Import LangChain Components

We'll use LangChain for LLM interaction and message handling.


In [39]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

print("✅ LangChain components imported")
print(f"   ChatOpenAI: Available")
print(f"   Message types: HumanMessage, SystemMessage, AIMessage")


✅ LangChain components imported
   ChatOpenAI: Available
   Message types: HumanMessage, SystemMessage, AIMessage


### Import Agent Memory Server Client

The Agent Memory Server provides production-ready memory management. If it's not available, we'll note that and continue with limited functionality.


In [40]:
# Import Agent Memory Server client
try:
    from agent_memory_client import MemoryAPIClient, MemoryClientConfig
    from agent_memory_client.models import WorkingMemory, MemoryMessage, ClientMemoryRecord
    MEMORY_SERVER_AVAILABLE = True
    print("✅ Agent Memory Server client available")
    print("   MemoryAPIClient: Ready")
    print("   Memory models: WorkingMemory, MemoryMessage, ClientMemoryRecord")
except ImportError:
    MEMORY_SERVER_AVAILABLE = False
    print("⚠️  Agent Memory Server not available")
    print("   Install with: pip install agent-memory-client")
    print("   Start server: See reference-agent/README.md")
    print("   Note: Some demos will be skipped")


✅ Agent Memory Server client available
   MemoryAPIClient: Ready
   Memory models: WorkingMemory, MemoryMessage, ClientMemoryRecord


### What We Just Did

We've successfully set up our environment with all the necessary components:

**Imported:**
- ✅ Section 2 RAG components (`CourseManager`, `redis_config`, models)
- ✅ LangChain for LLM interaction
- ✅ Agent Memory Server client (if available)

**Why This Matters:**
- Building on Section 2's foundation (not starting from scratch)
- Agent Memory Server provides scalable, persistent memory
- Same Redis University domain for consistency

---

## 🔧 Initialize Components

Now let's initialize the components we'll use throughout this notebook.


### Initialize Course Manager

The `CourseManager` handles course search and retrieval, just like in Section 2.


In [41]:
# Initialize Course Manager
course_manager = CourseManager()

print("✅ Course Manager initialized")
print("   Ready to search and retrieve courses")


✅ Course Manager initialized
   Ready to search and retrieve courses


### Initialize LLM

We'll use GPT-4o with temperature=0.0 for consistent, deterministic responses.


In [42]:
# Initialize LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0.0)



### Initialize Memory Client

If the Agent Memory Server is available, we'll initialize the memory client. This client handles both working memory (conversation history) and long-term memory (persistent facts).


In [43]:
# Initialize Memory Client
if MEMORY_SERVER_AVAILABLE:
    config = MemoryClientConfig(
        base_url=AGENT_MEMORY_URL,
        default_namespace="redis_university"
    )
    memory_client = MemoryAPIClient(config=config)
    print("✅ Memory Client initialized")
    print(f"   Base URL: {config.base_url}")
    print(f"   Namespace: {config.default_namespace}")
    print("   Ready for working memory and long-term memory operations")
else:
    memory_client = None
    print("⚠️  Memory Server not available")
    print("   Running with limited functionality")
    print("   Some demos will be skipped")


✅ Memory Client initialized
   Base URL: http://localhost:8088
   Namespace: redis_university
   Ready for working memory and long-term memory operations


### Create Sample Student Profile

We'll create a sample student profile to use throughout our demos. This follows the same pattern from Section 2.


In [44]:
# Create sample student profile
sarah = StudentProfile(
    name="Sarah Chen",
    email="sarah.chen@university.edu",
    major="Computer Science",
    year=2,
    interests=["machine learning", "data science", "algorithms"],
    completed_courses=["CS101", "CS201"],
    current_courses=["MATH301"],
    preferred_format=CourseFormat.ONLINE,
    preferred_difficulty=DifficultyLevel.INTERMEDIATE
)

print("✅ Student profile created")
print(f"   Name: {sarah.name}")
print(f"   Major: {sarah.major}")
print(f"   Year: {sarah.year}")
print(f"   Interests: {', '.join(sarah.interests)}")
print(f"   Completed: {', '.join(sarah.completed_courses)}")
print(f"   Preferred Format: {sarah.preferred_format.value}")


✅ Student profile created
   Name: Sarah Chen
   Major: Computer Science
   Year: 2
   Interests: machine learning, data science, algorithms
   Completed: CS101, CS201
   Preferred Format: online


In [45]:
print("🎯 INITIALIZATION SUMMARY")
print(f"\n✅ Course Manager: Ready")
print(f"✅ LLM (GPT-4o): Ready")
print(f"{'✅' if MEMORY_SERVER_AVAILABLE else '⚠️ '} Memory Client: {'Ready' if MEMORY_SERVER_AVAILABLE else 'Not Available'}")
print(f"✅ Student Profile: {sarah.name}")


🎯 INITIALIZATION SUMMARY

✅ Course Manager: Ready
✅ LLM (GPT-4o): Ready
✅ Memory Client: Ready
✅ Student Profile: Sarah Chen


### Initialization Done
📋 What We're Building On:
-  Section 2's RAG foundation (CourseManager, redis_config)
-  Same StudentProfile model
-  Same Redis configuration

✨ What We're Adding:
-  Memory Client for conversation history
-  Working Memory for session context
-  Long-term Memory for persistent knowledge


---

## 📚 Part 1: Working Memory Fundamentals

### **What is Working Memory?**

Working memory stores **conversation messages** for the current session. It enables:

- ✅ **Reference resolution** - "it", "that course", "the one you mentioned"
- ✅ **Context continuity** - Each message builds on previous messages
- ✅ **Natural conversations** - Users don't repeat themselves

### **How It Works:**

```
Turn 1: Load working memory (empty) → Process query → Save messages
Turn 2: Load working memory (1 exchange) → Process query → Save messages
Turn 3: Load working memory (2 exchanges) → Process query → Save messages
```

Each turn has access to all previous messages in the session.

---

## 🧪 Hands-On: Working Memory in Action

Let's simulate a multi-turn conversation with working memory. We'll break this down step-by-step to see how working memory enables natural conversation flow.


### Setup: Create Session and Student IDs

Now that we have our components initialized, let's create session and student identifiers for our working memory demo.


In [46]:
# Setup for working memory demo
student_id = sarah.email.split('@')[0]  # "sarah.chen"
session_id = f"session_{student_id}_demo"

print("🎯 Working Memory Demo Setup")
print(f"   Student ID: {student_id}")
print(f"   Session ID: {session_id}")
print("   Ready to demonstrate multi-turn conversation")


🎯 Working Memory Demo Setup
   Student ID: sarah.chen
   Session ID: session_sarah.chen_demo
   Ready to demonstrate multi-turn conversation


### Turn 1: Initial Query

Let's start with a simple query about a course. This is the first turn, so working memory will be empty.

We'll break this down into clear steps:
1. We will use Memory Server
2. Load working memory (will be empty on first turn)
3. Search for the course
4. Generate a response
5. Save the conversation to working memory


#### Step 1: Set up the user query


In [72]:
# Check if Memory Server is available

print("=" * 80)
print("📍 TURN 1: User asks about a course")
print("=" * 80)

# Define the user's query
turn1_query = "Tell me about Data Structures and Algorithms"
print(f"\n👤 User: {turn1_query}")


📍 TURN 1: User asks about a course

👤 User: Tell me about Data Structures and Algorithms


#### Step 2: Load working memory

On the first turn, working memory will be empty since this is a new session.


In [73]:
# Load working memory (empty for first turn)
_, turn1_working_memory = await memory_client.get_or_create_working_memory(
    session_id=session_id,
    user_id=student_id,
    model_name="gpt-4o"
)

print(f"📊 Working Memory Status:")
print(f"   Messages in memory: {len(turn1_working_memory.messages)}")
print(f"   Status: {'Empty (first turn)' if len(turn1_working_memory.messages) == 0 else 'Has history'}")


12:07:59 httpx INFO   HTTP Request: GET http://localhost:8088/v1/working-memory/session_sarah.chen_demo?user_id=sarah.chen&namespace=redis_university&model_name=gpt-4o "HTTP/1.1 200 OK"
📊 Working Memory Status:
   Messages in memory: 2
   Status: Has history


In [74]:
# observe the object
turn1_working_memory

WorkingMemoryResponse(messages=[MemoryMessage(role='user', content='Tell me about CS401', id='01K8XF2FBC4YDC5QNVQ8ZQKXNC', created_at=datetime.datetime(2025, 10, 31, 15, 44, 39, 788221, tzinfo=TzInfo(0)), persisted_at=None, discrete_memory_extracted='f'), MemoryMessage(role='assistant', content='CS009: Data Structures and Algorithms. Study of fundamental data structures and algorithms. Arrays, linked lists, trees, graphs, sorting, a...', id='01K8XF2FBC4YDC5QNVQ8ZQKXND', created_at=datetime.datetime(2025, 10, 31, 15, 44, 39, 788242, tzinfo=TzInfo(0)), persisted_at=None, discrete_memory_extracted='f')], memories=[], data={}, context=None, user_id='sarah.chen', tokens=0, session_id='session_sarah.chen_demo', namespace='redis_university', long_term_memory_strategy=MemoryStrategyConfig(strategy='discrete', config={}), ttl_seconds=None, last_accessed=datetime.datetime(2025, 10, 31, 15, 44, 39, tzinfo=TzInfo(0)), context_percentage_total_used=0.0296875, context_percentage_until_summarization=

#### Step 3: Search for the course

Use the course manager to search for courses matching the query.


In [75]:
print(f"\n🔍 Searching for courses...")
turn1_courses = await course_manager.search_courses(turn1_query, limit=1)

if turn1_courses:
    print(f"   Found {len(turn1_courses)} course(s)")

    # print the course details
    for course in turn1_courses:
        print(f"   - {course.course_code}: {course.title}")


🔍 Searching for courses...
12:08:01 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
   Found 1 course(s)
   - CS009: Data Structures and Algorithms


#### Step 4: Generate response using LLM

Use the LLM to generate a natural response based on the retrieved course information.

This follows the **RAG pattern**: Retrieve (done in Step 3) → Augment (add to context) → Generate (use LLM).


In [84]:
course = turn1_courses[0]

course_context = f"""Course Information:
- Code: {course.course_code}
- Title: {course.title}
- Description: {course.description}
- Prerequisites: {', '.join([p.course_code for p in course.prerequisites]) if course.prerequisites else 'None'}
- Credits: {course.credits}
"""

print(f"   Course context: {course_context}")

   Course context: Course Information:
- Code: CS009
- Title: Data Structures and Algorithms
- Description: Study of fundamental data structures and algorithms. Arrays, linked lists, trees, graphs, sorting, and searching.
- Prerequisites: CS001, CS001
- Credits: 4



In [85]:
# Build messages for LLM
turn1_messages = [
    SystemMessage(content="You are a helpful course advisor. Answer questions about courses based on the provided information."),
    HumanMessage(content=f"{course_context}\n\nUser question: {turn1_query}")
]

# Generate response using LLM
print(f"\n💭 Generating response using LLM...")
turn1_response = llm.invoke(turn1_messages).content

print(f"\n🤖 Agent: {turn1_response}")


💭 Generating response using LLM...
12:11:03 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

🤖 Agent: The course "Data Structures and Algorithms" (CS009) is a 4-credit course that focuses on the study of fundamental data structures and algorithms. In this course, you will learn about various data structures such as arrays, linked lists, trees, and graphs. Additionally, the course covers essential algorithms related to sorting and searching. 

To enroll in this course, you must have completed the prerequisite course CS001. This foundational knowledge will help you understand and apply the concepts taught in CS009 effectively.


#### Step 5: Save to working memory

Add both the user query and assistant response to working memory for future turns.


In [86]:
if MEMORY_SERVER_AVAILABLE:
    # Add messages to working memory
    turn1_working_memory.messages.extend([
        MemoryMessage(role="user", content=turn1_query),
        MemoryMessage(role="assistant", content=turn1_response)
    ])

    # Save to Memory Server
    await memory_client.put_working_memory(
        session_id=session_id,
        memory=turn1_working_memory,
        user_id=student_id,
        model_name="gpt-4o"
    )

    print(f"\n✅ Saved to working memory")
    print(f"   Messages now in memory: {len(turn1_working_memory.messages)}")


12:11:06 httpx INFO   HTTP Request: PUT http://localhost:8088/v1/working-memory/session_sarah.chen_demo?user_id=sarah.chen&model_name=gpt-4o "HTTP/1.1 200 OK"

✅ Saved to working memory
   Messages now in memory: 6


### What Just Happened in Turn 1?

**Initial State:**
- Working memory was empty (first turn)
- No conversation history available

**Actions (RAG Pattern):**
1. **Retrieve:** Searched for Data Structures and Algorithms in the course database
2. **Augment:** Added course information to LLM context
3. **Generate:** LLM created a natural language response
4. **Save:** Stored conversation in working memory

**Result:**
- Working memory now contains 2 messages (1 user, 1 assistant)
- This history will be available for the next turn

**Key Insight:** Even the first turn uses the LLM to generate natural responses based on retrieved information.

---


### Turn 2: Follow-up with Pronoun Reference

Now let's ask a follow-up question using "its" - a pronoun that requires context from Turn 1.

We'll break this down into steps:
1. Set up the query with pronoun reference
2. Load working memory (now contains Turn 1)
3. Build context with conversation history
4. Generate response using LLM
5. Save to working memory


#### Step 1: Set up the query


In [87]:
if MEMORY_SERVER_AVAILABLE:
    print("\n" + "=" * 80)
    print("📍 TURN 2: User uses pronoun reference ('its')")
    print("=" * 80)

    turn2_query = "What are its prerequisites?"
    print(f"\n👤 User: {turn2_query}")
    print(f"   Note: 'its' refers to Data Structures and Algorithms from Turn 1")



📍 TURN 2: User uses pronoun reference ('its')

👤 User: What are its prerequisites?
   Note: 'its' refers to Data Structures and Algorithms from Turn 1


#### Step 2: Load working memory

This time, working memory will contain the conversation from Turn 1.


In [88]:
if MEMORY_SERVER_AVAILABLE:
    # Load working memory (now has 1 exchange from Turn 1)
    _, turn2_working_memory = await memory_client.get_or_create_working_memory(
        session_id=session_id,
        user_id=student_id,
        model_name="gpt-4o"
    )

    print(f"\n📊 Working Memory Status:")
    print(f"   Messages in memory: {len(turn2_working_memory.messages)}")
    print(f"   Contains: Turn 1 conversation")


12:11:12 httpx INFO   HTTP Request: GET http://localhost:8088/v1/working-memory/session_sarah.chen_demo?user_id=sarah.chen&namespace=redis_university&model_name=gpt-4o "HTTP/1.1 200 OK"

📊 Working Memory Status:
   Messages in memory: 6
   Contains: Turn 1 conversation


#### Step 3: Build context with conversation history

To resolve the pronoun "its", we need to include the conversation history in the LLM context.


In [89]:
if MEMORY_SERVER_AVAILABLE:
    print(f"\n🔧 Building context with conversation history...")

    # Start with system message
    turn2_messages = [
        SystemMessage(content="You are a helpful course advisor. Use conversation history to resolve references like 'it', 'that course', etc.")
    ]

    # Add conversation history from working memory
    for msg in turn2_working_memory.messages:
        if msg.role == "user":
            turn2_messages.append(HumanMessage(content=msg.content))
        elif msg.role == "assistant":
            turn2_messages.append(AIMessage(content=msg.content))

    # Add current query
    turn2_messages.append(HumanMessage(content=turn2_query))

    print(f"   Total messages in context: {len(turn2_messages)}")
    print(f"   Includes: System prompt + Turn 1 history + current query")



🔧 Building context with conversation history...
   Total messages in context: 8
   Includes: System prompt + Turn 1 history + current query


#### Step 4: Generate response using LLM

The LLM can now resolve "its" by looking at the conversation history.


In [90]:
if MEMORY_SERVER_AVAILABLE:
    print(f"\n💭 LLM resolving 'its' using conversation history...")
    turn2_response = llm.invoke(turn2_messages).content

    print(f"\n🤖 Agent: {turn2_response}")



💭 LLM resolving 'its' using conversation history...
12:11:18 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

🤖 Agent: The prerequisite for the "Data Structures and Algorithms" course (CS009) is CS001. You need to have completed CS001 to enroll in CS009, as it provides the foundational knowledge necessary for understanding the more advanced concepts covered in the course.


#### Step 5: Save to working memory

Add this turn's conversation to working memory for future turns.


In [91]:
if MEMORY_SERVER_AVAILABLE:
    # Add messages to working memory
    turn2_working_memory.messages.extend([
        MemoryMessage(role="user", content=turn2_query),
        MemoryMessage(role="assistant", content=turn2_response)
    ])

    # Save to Memory Server
    await memory_client.put_working_memory(
        session_id=session_id,
        memory=turn2_working_memory,
        user_id=student_id,
        model_name="gpt-4o"
    )

    print(f"\n✅ Saved to working memory")
    print(f"   Messages now in memory: {len(turn2_working_memory.messages)}")


12:11:30 httpx INFO   HTTP Request: PUT http://localhost:8088/v1/working-memory/session_sarah.chen_demo?user_id=sarah.chen&model_name=gpt-4o "HTTP/1.1 200 OK"

✅ Saved to working memory
   Messages now in memory: 8


### What Just Happened in Turn 2?

**Initial State:**
- Working memory contained Turn 1 conversation (2 messages)
- User asked about "its prerequisites" - pronoun reference

**Actions:**
1. Loaded working memory with Turn 1 history
2. Built context including conversation history
3. LLM resolved "its" → Data Structures and Algorithms (from Turn 1)
4. Generated response about Data Structures and Algorithms's prerequisites
5. Saved updated conversation to working memory

**Result:**
- Working memory now contains 4 messages (2 exchanges)
- LLM successfully resolved pronoun reference using conversation history
- Natural conversation flow maintained

**Key Insight:** Without working memory, the LLM wouldn't know what "its" refers to!

---


### Turn 3: Another Follow-up

Let's ask one more follow-up question to demonstrate continued conversation continuity.


#### Step 1: Set up the query


In [92]:
if MEMORY_SERVER_AVAILABLE:
    print("\n" + "=" * 80)
    print("📍 TURN 3: User asks another follow-up")
    print("=" * 80)

    turn3_query = "Can I take it next semester?"
    print(f"\n👤 User: {turn3_query}")
    print(f"   Note: 'it' refers to Data Structures and Algorithms from Turn 1")



📍 TURN 3: User asks another follow-up

👤 User: Can I take it next semester?
   Note: 'it' refers to Data Structures and Algorithms from Turn 1


#### Step 2: Load working memory with full conversation history


In [93]:
if MEMORY_SERVER_AVAILABLE:
    # Load working memory (now has 2 exchanges)
    _, turn3_working_memory = await memory_client.get_or_create_working_memory(
        session_id=session_id,
        user_id=student_id,
        model_name="gpt-4o"
    )

    print(f"\n📊 Working Memory Status:")
    print(f"   Messages in memory: {len(turn3_working_memory.messages)}")
    print(f"   Contains: Turns 1 and 2")


12:12:55 httpx INFO   HTTP Request: GET http://localhost:8088/v1/working-memory/session_sarah.chen_demo?user_id=sarah.chen&namespace=redis_university&model_name=gpt-4o "HTTP/1.1 200 OK"

📊 Working Memory Status:
   Messages in memory: 8
   Contains: Turns 1 and 2


#### Step 3: Build context and generate response


In [94]:
if MEMORY_SERVER_AVAILABLE:
    # Build context with full conversation history
    turn3_messages = [
        SystemMessage(content="You are a helpful course advisor. Use conversation history to resolve references.")
    ]

    for msg in turn3_working_memory.messages:
        if msg.role == "user":
            turn3_messages.append(HumanMessage(content=msg.content))
        elif msg.role == "assistant":
            turn3_messages.append(AIMessage(content=msg.content))

    turn3_messages.append(HumanMessage(content=turn3_query))

    print(f"   Total messages in context: {len(turn3_messages)}")

    # Generate response
    turn3_response = llm.invoke(turn3_messages).content

    print(f"\n🤖 Agent: {turn3_response}")


   Total messages in context: 10
12:13:14 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

🤖 Agent: To determine if you can take "Data Structures and Algorithms" (CS009) next semester, you'll need to check the course schedule for the upcoming semester at your institution. Ensure that you have completed the prerequisite course, CS001, before enrolling. If you meet the prerequisite and the course is offered, you should be able to register for it. It's also a good idea to consult with your academic advisor to confirm your eligibility and to help with planning your course schedule.




✅ DEMO COMPLETE: Working memory enabled natural conversation flow!

---
### Working Memory Demo Summary

Let's review what we just demonstrated across three conversation turns.

## 🎯 Working Memory Demo Summary
### 📊 What Happened:
**Turn 1:** 'Tell me about Data Structures and Algorithms'
- Working memory: empty (first turn)
- Stored query and response

**Turn 2:** 'What are its prerequisites?'
- Working memory: 1 exchange (Turn 1)
- LLM resolved 'its' → Data Structures and Algorithms using history
- Generated accurate response

**Turn 3:** 'Can I take it next semester?'
- Working memory: 2 exchanges (Turns 1-2)
- LLM resolved 'it' → Data Structures and Algorithms using history
- Maintained conversation continuity

#### ✅ Key Benefits:
- Natural conversation flow
- Pronoun reference resolution
- No need to repeat context
- Seamless user experience

#### ❌ Without Working Memory:
- 'What are its prerequisites?' → 'What is its?' Or "General information without data from the LLM's training"
- Each query is isolated
- User must repeat context every time

### Key Insight: Conversation Context Type

Working memory provides the **Conversation Context** - the third context type from Section 1:

1. **System Context** - Role and instructions (static)
2. **User Context** - Profile and preferences (dynamic, user-specific)
3. **Conversation Context** - Working memory (dynamic, session-specific) ← **We just demonstrated this!**
4. **Retrieved Context** - RAG results (dynamic, query-specific)

Without working memory, we only had 3 context types. Now we have all 4!


---
# 📚 Long-term Memory for Context Engineering

## What is Long-term Memory?

Long-term memory enables AI agents to store **persistent facts, preferences, and goals** across sessions. This is crucial for context engineering because it allows agents to:

- **Personalize** interactions by remembering user preferences
- **Accumulate knowledge** about users over time
- **Maintain continuity** across multiple conversations
- **Search efficiently** using semantic vector search

### How It Works

```
Session 1: User shares preferences → Store in long-term memory
Session 2: User asks for recommendations → Search memory → Personalized response
Session 3: User updates preferences → Update memory accordingly
```

---

## Three Types of Long-term Memory

The Agent Memory Server supports three distinct memory types, each optimized for different kinds of information:

### 1. Semantic Memory - Facts and Knowledge

**Purpose:** Store timeless facts, preferences, and knowledge independent of when they were learned.

**Examples:**
- "Student's major is Computer Science"
- "Student prefers online courses"
- "Student wants to graduate in Spring 2026"
- "Student is interested in machine learning"

**When to use:** Information that remains true regardless of time context.

---

### 2. Episodic Memory - Events and Experiences

**Purpose:** Store time-bound events and experiences where sequence matters.

**Examples:**
- "Student enrolled in CS101 on 2024-09-15"
- "Student completed CS101 with grade A on 2024-12-10"
- "Student asked about machine learning courses on 2024-09-20"

**When to use:** Timeline-based information where timing or sequence is important.

---

### 3. Message Memory - Context-Rich Conversations

**Purpose:** Store full conversation snippets where complete context is crucial.

**Examples:**
- Detailed career planning discussion with nuanced advice
- Professor's specific guidance about research opportunities
- Student's explanation of personal learning challenges

**When to use:** When summary would lose important nuance, tone, or exact wording.

**⚠️ Use sparingly** - Message memories are token-expensive!

---

## 🎯 Choosing the Right Memory Type

### Decision Framework

**Ask yourself these questions:**

1. **Can you extract a simple fact?** → Use **Semantic**
2. **Does timing matter?** → Use **Episodic**
3. **Is full context crucial?** → Use **Message** (rarely)

**Default strategy: Prefer Semantic** - they're compact, searchable, and efficient.

---

### Quick Reference Table

| Information Type | Memory Type | Example |
|-----------------|-------------|----------|
| Preference | Semantic | "Prefers morning classes" |
| Fact | Semantic | "Major is Computer Science" |
| Goal | Semantic | "Wants to graduate in 2026" |
| Event | Episodic | "Enrolled in CS401 on 2024-09-15" |
| Timeline | Episodic | "Completed CS101, then CS201" |
| Complex discussion | Message | [Full career planning conversation] |
| Nuanced advice | Message | [Professor's detailed guidance] |

---

## Examples: Right vs. Wrong Choices

### Scenario 1: Student States Preference

**User says:** "I prefer online courses because I work during the day."

❌ **Wrong - Message memory (too verbose):**
```python
memory = "Student said: 'I prefer online courses because I work during the day.'"
```

✅ **Right - Semantic memories (extracted facts):**
```python
memory1 = "Student prefers online courses"
memory2 = "Student works during the day"
```

**Why:** Simple facts don't need verbatim storage.

---

### Scenario 2: Course Completion

**User says:** "I just finished CS101 last week!"

❌ **Wrong - Semantic (loses temporal context):**
```python
memory = "Student completed CS101"
```

✅ **Right - Episodic (preserves timeline):**
```python
memory = "Student completed CS101 on 2024-10-20"
```

**Why:** Timeline matters for prerequisites and future planning.

---

### Scenario 3: Complex Career Advice

**Context:** 20-message discussion about career path including nuanced advice about research vs. industry, application timing, and specific companies to target.

❌ **Wrong - Semantic (loses too much context):**
```python
memory = "Student discussed career planning"
```

✅ **Right - Message memory (preserves full context):**
```python
memory = [Full conversation thread with all nuance]
```

**Why:** Details and context are critical; summary would be inadequate.

---

## Key Takeaways

- **Most memories should be semantic** - efficient and searchable
- **Use episodic when sequence matters** - track progress and timeline
- **Use message rarely** - only when context cannot be summarized
- **Effective memory selection improves personalization** and reduces token usage

---

## 🧪 Hands-On: Long-term Memory in Action

Let's put these concepts into practice with code examples...

### Setup: Student ID for Long-term Memory

Long-term memories are user-scoped, so we need a student ID.


In [1]:
# Setup for long-term memory demo
lt_student_id = "sarah_chen"

print("🎯 Long-term Memory Demo Setup")
print(f"   Student ID: {lt_student_id}")
print("   Ready to store and search persistent memories")


🎯 Long-term Memory Demo Setup
   Student ID: sarah_chen
   Ready to store and search persistent memories


### Step 1: Store Semantic Memories (Facts)

Semantic memories are timeless facts about the student. Let's store several facts about Sarah's preferences and academic status.


In [ ]:
# Step 1: Store semantic memories
async def store_semantic_memories():
    """Store semantic memories (facts) about the student"""

    if not MEMORY_SERVER_AVAILABLE:
        print("⚠️  Memory Server not available. Skipping demo.")
        return

    print("=" * 80)
    print("📍 STEP 1: Storing Semantic Memories (Facts)")
    print("=" * 80)

    semantic_memories = [
        "Student prefers online courses over in-person classes",
        "Student's major is Computer Science with focus on AI/ML",
        "Student wants to graduate in Spring 2026",
        "Student prefers morning classes, no classes on Fridays",
        "Student has completed CS101 and CS201",
        "Student is currently taking MATH301"
    ]

    print(f"\n📝 Storing {len(semantic_memories)} semantic memories...")

    for memory_text in semantic_memories:
        memory_record = ClientMemoryRecord(
            text=memory_text,
            user_id=lt_student_id,
            memory_type="semantic",
            topics=["preferences", "academic_info"]
        )
        await memory_client.create_long_term_memory([memory_record])
        print(f"   ✅ {memory_text}")

    print(f"\n✅ Stored {len(semantic_memories)} semantic memories")
    print("   Memory type: semantic (timeless facts)")
    print("   Topics: preferences, academic_info")

# Run Step 1
await store_semantic_memories()


### What We Just Did: Semantic Memories

**Stored 6 semantic memories:**
- Student preferences (online courses, morning classes)
- Academic information (major, graduation date)
- Course history (completed, current)

**Why semantic?**
- These are timeless facts
- No specific date/time context needed
- Compact and efficient

**How they're stored:**
- Vector-indexed for semantic search
- Tagged with topics for organization
- Automatically deduplicated

---


### Step 2: Store Episodic Memories (Events)

Episodic memories are time-bound events. Let's store some events from Sarah's academic timeline.


In [ ]:
# Step 2: Store episodic memories
async def store_episodic_memories():
    """Store episodic memories (events) about the student"""

    if not MEMORY_SERVER_AVAILABLE:
        print("⚠️  Memory Server not available. Skipping demo.")
        return

    print("\n" + "=" * 80)
    print("📍 STEP 2: Storing Episodic Memories (Events)")
    print("=" * 80)

    episodic_memories = [
        "Student enrolled in CS101 on 2024-09-01",
        "Student completed CS101 with grade A on 2024-12-15",
        "Student asked about machine learning courses on 2024-09-20"
    ]

    print(f"\n📝 Storing {len(episodic_memories)} episodic memories...")

    for memory_text in episodic_memories:
        memory_record = ClientMemoryRecord(
            text=memory_text,
            user_id=lt_student_id,
            memory_type="episodic",
            topics=["enrollment", "courses"]
        )
        await memory_client.create_long_term_memory([memory_record])
        print(f"   ✅ {memory_text}")

    print(f"\n✅ Stored {len(episodic_memories)} episodic memories")
    print("   Memory type: episodic (time-bound events)")
    print("   Topics: enrollment, courses")

# Run Step 2
await store_episodic_memories()


### What We Just Did: Episodic Memories

**Stored 3 episodic memories:**
- Enrollment event (CS101 on 2024-09-01)
- Completion event (CS101 with grade A on 2024-12-15)
- Interaction event (asked about ML courses on 2024-09-20)

**Why episodic?**
- These are time-bound events
- Timing and sequence matter
- Captures academic timeline

**Difference from semantic:**
- Semantic: "Student has completed CS101" (timeless fact)
- Episodic: "Student completed CS101 with grade A on 2024-12-15" (specific event)

---


### Step 3: Search Long-term Memory

Now let's search our long-term memories using natural language queries. The system will use semantic search to find relevant memories.


In [ ]:
# Step 3: Search long-term memory
async def search_longterm_memories():
    """Search long-term memory with semantic queries"""

    if not MEMORY_SERVER_AVAILABLE:
        print("⚠️  Memory Server not available. Skipping demo.")
        return

    print("\n" + "=" * 80)
    print("📍 STEP 3: Searching Long-term Memory")
    print("=" * 80)

    search_queries = [
        "What does the student prefer?",
        "What courses has the student completed?",
        "What is the student's major?"
    ]

    for query in search_queries:
        print(f"\n🔍 Query: '{query}'")
        results = await memory_client.search_long_term_memory(
            text=query,
            user_id=lt_student_id,
            limit=3
        )

        if results.memories:
            print(f"   📚 Found {len(results.memories)} relevant memories:")
            for i, memory in enumerate(results.memories[:3], 1):
                print(f"      {i}. {memory.text}")
        else:
            print("   ⚠️  No memories found")

    print("\n" + "=" * 80)
    print("✅ DEMO COMPLETE: Long-term memory enables persistent knowledge!")
    print("=" * 80)

# Run Step 3
await search_longterm_memories()


### Long-term Memory Demo Summary

Let's review what we demonstrated with long-term memory.


In [ ]:
print("=" * 80)
print("🎯 LONG-TERM MEMORY DEMO SUMMARY")
print("=" * 80)
print("\n📊 What We Did:")
print("   Step 1: Stored 6 semantic memories (facts)")
print("           → Student preferences, major, graduation date")
print("           → Tagged with topics: preferences, academic_info")
print("\n   Step 2: Stored 3 episodic memories (events)")
print("           → Enrollment, completion, interaction events")
print("           → Tagged with topics: enrollment, courses")
print("\n   Step 3: Searched long-term memory")
print("           → Used natural language queries")
print("           → Semantic search found relevant memories")
print("           → No exact keyword matching needed")
print("\n✅ Key Benefits:")
print("   • Persistent knowledge across sessions")
print("   • Semantic search (not keyword matching)")
print("   • Automatic deduplication")
print("   • Topic-based organization")
print("\n💡 Key Insight:")
print("   Long-term memory enables personalization and knowledge")
print("   accumulation across sessions. It's the foundation for")
print("   building agents that remember and learn from users.")
print("=" * 80)


### Key Insight: User Context Type

Long-term memory provides part of the **User Context** - the second context type from Section 1:

1. **System Context** - Role and instructions (static)
2. **User Context** - Profile + long-term memories (dynamic, user-specific) ← **Long-term memories contribute here!**
3. **Conversation Context** - Working memory (dynamic, session-specific)
4. **Retrieved Context** - RAG results (dynamic, query-specific)

Long-term memories enhance User Context by adding persistent knowledge about the user's preferences, history, and goals.

---

## 🏷️ Advanced: Topics and Filtering

Topics help organize and filter memories. Let's explore how to use them effectively.


### Step 1: Store memories with topics


In [ ]:
if MEMORY_SERVER_AVAILABLE:
    topics_student_id = "sarah_chen"

    print("=" * 80)
    print("🏷️  TOPICS AND FILTERING DEMO")
    print("=" * 80)

    print("\n📍 Storing Memories with Topics")
    print("-" * 80)

    # Define memories with their topics
    memories_with_topics = [
        ("Student prefers online courses", ["preferences", "course_format"]),
        ("Student's major is Computer Science", ["academic_info", "major"]),
        ("Student wants to graduate in Spring 2026", ["goals", "graduation"]),
        ("Student prefers morning classes", ["preferences", "schedule"]),
    ]

    # Store each memory
    for memory_text, topics in memories_with_topics:
        memory_record = ClientMemoryRecord(
            text=memory_text,
            user_id=topics_student_id,
            memory_type="semantic",
            topics=topics
        )
        await memory_client.create_long_term_memory([memory_record])
        print(f"   ✅ {memory_text}")
        print(f"      Topics: {', '.join(topics)}")


### Step 2: Filter memories by type


In [ ]:
if MEMORY_SERVER_AVAILABLE:
    print("\n📍 Filtering by Memory Type: Semantic")
    print("-" * 80)

    from agent_memory_client.models import MemoryType

    # Search for all semantic memories
    results = await memory_client.search_long_term_memory(
        text="",  # Empty query returns all
        user_id=topics_student_id,
        memory_type=MemoryType(eq="semantic"),
        limit=10
    )

    print(f"   Found {len(results.memories)} semantic memories:")
    for i, memory in enumerate(results.memories[:5], 1):
        topics_str = ', '.join(memory.topics) if memory.topics else 'none'
        print(f"   {i}. {memory.text}")
        print(f"      Topics: {topics_str}")

    print("\n" + "=" * 80)
    print("✅ Topics enable organized, filterable memory management!")
    print("=" * 80)


### 🎯 Why Topics Matter

**Organization:**
- Group related memories together
- Easy to find memories by category

**Filtering:**
- Search within specific topics
- Filter by memory type (semantic, episodic, message)

**Best Practices:**
- Use consistent topic names
- Keep topics broad enough to be useful
- Common topics: `preferences`, `academic_info`, `goals`, `schedule`, `courses`

---

## 🔄 Cross-Session Memory Persistence

Let's verify that memories persist across sessions.


### Step 1: Session 1 - Store memories


In [ ]:
if MEMORY_SERVER_AVAILABLE:
    cross_session_student_id = "sarah_chen"

    print("=" * 80)
    print("🔄 CROSS-SESSION MEMORY PERSISTENCE DEMO")
    print("=" * 80)

    print("\n📍 SESSION 1: Storing Memories")
    print("-" * 80)

    memory_record = ClientMemoryRecord(
        text="Student is interested in machine learning and AI",
        user_id=cross_session_student_id,
        memory_type="semantic",
        topics=["interests", "AI"]
    )
    await memory_client.create_long_term_memory([memory_record])
    print("   ✅ Stored: Student is interested in machine learning and AI")


### Step 2: Session 2 - Create new client and retrieve memories

Simulate a new session by creating a new memory client.


In [ ]:
if MEMORY_SERVER_AVAILABLE:
    print("\n📍 SESSION 2: New Session, Same Student")
    print("-" * 80)

    # Create a new memory client (simulating a new session)
    new_session_config = MemoryClientConfig(
        base_url=os.getenv("AGENT_MEMORY_URL", "http://localhost:8000"),
        default_namespace="redis_university"
    )
    new_session_client = MemoryAPIClient(config=new_session_config)

    print("   🔄 New session started for the same student")

    # Search for memories from the new session
    print("\n   🔍 Searching: 'What are the student's interests?'")
    cross_session_results = await new_session_client.search_long_term_memory(
        text="What are the student's interests?",
        user_id=cross_session_student_id,
        limit=3
    )

    if cross_session_results.memories:
        print(f"\n   ✅ Memories accessible from new session:")
        for i, memory in enumerate(cross_session_results.memories[:3], 1):
            print(f"      {i}. {memory.text}")
    else:
        print("   ⚠️  No memories found")

    print("\n" + "=" * 80)
    print("✅ Long-term memories persist across sessions!")
    print("=" * 80)


### 🎯 Cross-Session Persistence

**What We Demonstrated:**
- **Session 1:** Stored memories about student interests
- **Session 2:** Created new client (simulating new session)
- **Result:** Memories from Session 1 are accessible in Session 2

**Why This Matters:**
- Users don't have to repeat themselves
- Personalization works across days, weeks, months
- Knowledge accumulates over time

**Contrast with Working Memory:**
- Working memory: Session-scoped (expires after 24 hours)
- Long-term memory: User-scoped (persists indefinitely)

---

## 🔗 What's Next: Memory-Enhanced RAG and Agents

You've learned the fundamentals of memory architecture! Now it's time to put it all together.

### **Next Notebook: `02_memory_enhanced_rag_and_agents.ipynb`**

In the next notebook, you'll:

1. **Build** a complete memory-enhanced RAG system
   - Integrate working memory + long-term memory + RAG
   - Combine all four context types
   - Show clear before/after comparisons

2. **Convert** to LangGraph agent (Part 2, separate notebook)
   - Add state management
   - Improve control flow
   - Prepare for Section 4 (tools and advanced capabilities)

**Why Continue?**
- See memory in action with real conversations
- Learn how to build production-ready agents
- Prepare for Section 4 (adding tools like enrollment, scheduling)

**📚 Continue to:** `02_memory_enhanced_rag_and_agents.ipynb`

## ⏰ Memory Lifecycle & Persistence

Understanding how long memories last and when they expire is crucial for building reliable systems.

### **Working Memory TTL (Time-To-Live)**

**Default TTL:** 24 hours

**What this means:**
- Working memory (conversation history) expires 24 hours after last activity
- After expiration, conversation context is lost
- Long-term memories extracted from the conversation persist

**Timeline Example:**

```
Day 1, 10:00 AM - Session starts
Day 1, 10:25 AM - Session ends
    ↓
[24 hours later]
    ↓
Day 2, 10:25 AM - Working memory still available ✅
Day 2, 10:26 AM - Working memory expires ❌
```

### **Long-term Memory Persistence**

**Lifetime:** Indefinite (until manually deleted)

**What this means:**
- Long-term memories never expire automatically
- Accessible across all sessions, forever
- Must be explicitly deleted if no longer needed

### **Why This Design?**

**Working Memory (Short-lived):**
- Conversations are temporary
- Most context is only relevant during the session
- Automatic cleanup prevents storage bloat
- Privacy: Old conversations don't linger

**Long-term Memory (Persistent):**
- Important facts should persist
- User preferences don't expire
- Knowledge accumulates over time
- Enables true personalization

### **Important Implications**

**1. Extract Before Expiration**

If something important is said in conversation, it must be extracted to long-term memory before the 24-hour TTL expires.

**Good news:** Agent Memory Server does this automatically!

**2. Long-term Memories are Permanent**

Once stored, long-term memories persist indefinitely. Be thoughtful about what you store.

**3. Cross-Session Behavior**

```
Session 1 (Day 1):
- User: "I'm interested in machine learning"
- Working memory: Stores conversation
- Long-term memory: Extracts "Student interested in machine learning"

[30 hours later - Working memory expired]

Session 2 (Day 3):
- Working memory from Session 1: EXPIRED ❌
- Long-term memory: Still available ✅
- Agent retrieves: "Student interested in machine learning"
- Agent makes relevant recommendations ✅
```

### **Practical Multi-Day Conversation Example**


In [ ]:
# Multi-Day Conversation Simulation
async def multi_day_simulation():
    """Simulate conversations across multiple days"""

    if not MEMORY_SERVER_AVAILABLE:
        print("⚠️  Memory Server not available. Skipping demo.")
        return

    student_id = "sarah_chen"

    print("=" * 80)
    print("⏰ MULTI-DAY CONVERSATION SIMULATION")
    print("=" * 80)

    # Day 1: Initial conversation
    print("\n📅 DAY 1: Initial Conversation")
    print("-" * 80)

    session_1 = f"session_{student_id}_day1"

    # Store a fact in long-term memory
    memory_record = ClientMemoryRecord(
        text="Student is preparing for a career in AI research",
        user_id=student_id,
        memory_type="semantic",
        topics=["career", "goals"]
    )
    await memory_client.create_long_term_memory([memory_record])
    print("   ✅ Stored in long-term memory: Career goal (AI research)")

    # Simulate working memory (would normally be conversation)
    print("   💬 Working memory: Active for session_day1")
    print("   ⏰ TTL: 24 hours from now")

    # Day 3: New conversation (working memory expired)
    print("\n📅 DAY 3: New Conversation (48 hours later)")
    print("-" * 80)

    session_2 = f"session_{student_id}_day3"

    print("   ❌ Working memory from Day 1: EXPIRED")
    print("   ✅ Long-term memory: Still available")

    # Search long-term memory
    results = await memory_client.search_long_term_memory(
        text="What are the student's career goals?",
        user_id=student_id,
        limit=3
    )

    if results.memories:
        print("\n   🔍 Retrieved from long-term memory:")
        for memory in results.memories[:3]:
            print(f"      • {memory.text}")
        print("\n   ✅ Agent can still personalize recommendations!")

    print("\n" + "=" * 80)
    print("✅ Long-term memories persist, working memory expires")
    print("=" * 80)

# Run the simulation
await multi_day_simulation()


### 🎯 Memory Lifecycle Best Practices

**1. Trust Automatic Extraction**
- Agent Memory Server automatically extracts important facts
- Don't manually store everything in long-term memory
- Let the system decide what's important

**2. Use Appropriate Memory Types**
- Working memory: Current conversation only
- Long-term memory: Facts that should persist

**3. Monitor Memory Growth**
- Long-term memories accumulate over time
- Implement cleanup for outdated information
- Consider archiving old memories

**4. Plan for Expiration**
- Working memory expires after 24 hours
- Important context must be in long-term memory
- Don't rely on working memory for cross-session data

**5. Test Cross-Session Behavior**
- Verify long-term memories are accessible
- Ensure personalization works after TTL expiration
- Test with realistic time gaps

---

## 🎓 Key Takeaways

### **1. Memory Solves the Grounding Problem**

Without memory, agents can't resolve references:
- ❌ "What are **its** prerequisites?" → Agent doesn't know what "its" refers to
- ✅ With working memory → Agent resolves "its" from conversation history

### **2. Two Types of Memory Serve Different Purposes**

**Working Memory (Session-Scoped):**
- Conversation messages from current session
- Enables reference resolution and conversation continuity
- TTL-based (expires after session ends)

**Long-term Memory (Cross-Session):**
- Persistent facts, preferences, goals
- Enables personalization across sessions
- Searchable via semantic vector search

### **3. Memory Completes the Four Context Types**

From Section 1, we learned about four context types. Memory enables two of them:

1. **System Context** (Static) - ✅ Section 2
2. **User Context** (Dynamic, User-Specific) - ✅ Section 2 + Long-term Memory
3. **Conversation Context** (Dynamic, Session-Specific) - ✨ **Working Memory**
4. **Retrieved Context** (Dynamic, Query-Specific) - ✅ Section 2 RAG

### **4. Memory + RAG = Complete Context Engineering**

The integration pattern:
```
1. Load working memory (conversation history)
2. Search long-term memory (user facts)
3. RAG search (relevant documents)
4. Assemble all context types
5. Generate response
6. Save working memory (updated conversation)
```

This gives us **stateful, personalized, context-aware conversations**.

### **5. Agent Memory Server is Production-Ready**

Why use Agent Memory Server instead of simple in-memory storage:
- ✅ **Scalable** - Redis-backed, handles thousands of users
- ✅ **Automatic** - Extracts important facts to long-term storage
- ✅ **Semantic search** - Vector-indexed memory retrieval
- ✅ **Deduplication** - Prevents redundant memories
- ✅ **TTL management** - Automatic expiration of old sessions

### **6. LangChain is Sufficient for Memory + RAG**

We didn't need LangGraph for this section because:
- Simple linear flow (load → search → generate → save)
- No conditional branching or complex state management
- No tool calling required

**LangGraph becomes necessary in Section 4** when we add tools and multi-step workflows.

### **7. Memory Management Best Practices**

**Choose the Right Memory Type:**
- **Semantic** for facts and preferences (most common)
- **Episodic** for time-bound events and timeline
- **Message** for context-rich conversations (use sparingly)

**Understand Memory Lifecycle:**
- **Working memory:** 24-hour TTL, session-scoped
- **Long-term memory:** Indefinite persistence, user-scoped
- **Automatic extraction:** Trust the system to extract important facts

**Benefits of Proper Memory Management:**
- ✅ **Natural conversations** - Users don't repeat themselves
- ✅ **Cross-session personalization** - Knowledge persists over time
- ✅ **Efficient storage** - Automatic deduplication prevents bloat
- ✅ **Semantic search** - Find relevant memories without exact keywords
- ✅ **Scalable** - Redis-backed, production-ready architecture

**Key Principle:** Memory transforms stateless RAG into stateful, personalized, context-aware conversations.

---

## 🚀 What's Next?

### **Next Notebook: Memory-Enhanced RAG and Agents**

**📚 Continue to: `02_memory_enhanced_rag_and_agents.ipynb`**

In the next notebook, you'll:

1. **Build** a complete memory-enhanced RAG system
   - Integrate working memory + long-term memory + RAG
   - Combine all four context types
   - Show clear before/after comparisons

2. **Convert** to LangGraph agent (Part 2, separate notebook)
   - Add state management
   - Improve control flow
   - Prepare for Section 4 (tools and advanced capabilities)

### **Then: Section 4 - Tools and Advanced Agents**

After completing the next notebook, you'll be ready for Section 4:

**Tools You'll Add:**
- `search_courses` - Semantic search
- `get_course_details` - Fetch specific course information
- `check_prerequisites` - Verify student eligibility
- `enroll_course` - Register student for a course
- `store_memory` - Explicitly save important facts

**The Complete Learning Path:**

```
Section 1: Context Engineering Fundamentals
    ↓
Section 2: RAG (Retrieved Context)
    ↓
Section 3 (Notebook 1): Memory Fundamentals ← You are here
    ↓
Section 3 (Notebook 2): Memory-Enhanced RAG and Agents
    ↓
Section 4: Tools + Agents (Complete Agentic System)
```

---

## 💪 Practice Exercises

### **Exercise 1: Cross-Session Personalization**

Modify the `memory_enhanced_rag_query` function to:
1. Store user preferences in long-term memory when mentioned
2. Use those preferences in future sessions
3. Test with two different sessions for the same student

**Hint:** Look for phrases like "I prefer...", "I like...", "I want..." and store them as semantic memories.

### **Exercise 2: Memory-Aware Filtering**

Enhance the RAG search to use long-term memories as filters:
1. Search long-term memory for preferences (format, difficulty, schedule)
2. Apply those preferences as filters to `course_manager.search_courses()`
3. Compare results with and without memory-aware filtering

**Hint:** Use the `filters` parameter in `course_manager.search_courses()`.

### **Exercise 3: Conversation Summarization**

Implement a function that summarizes long conversations:
1. When working memory exceeds 10 messages, summarize the conversation
2. Store the summary in long-term memory
3. Clear old messages from working memory (keep only recent 4)
4. Test that reference resolution still works with summarized history

**Hint:** Use the LLM to generate summaries, then store as semantic memories.

### **Exercise 4: Multi-User Memory Management**

Create a simple CLI that:
1. Supports multiple students (different user IDs)
2. Maintains separate working memory per session
3. Maintains separate long-term memory per user
4. Demonstrates cross-session continuity for each user

**Hint:** Use different `session_id` and `user_id` for each student.

### **Exercise 5: Memory Search Quality**

Experiment with long-term memory search:
1. Store 20+ diverse memories for a student
2. Try different search queries
3. Analyze which memories are retrieved
4. Adjust memory text to improve search relevance

**Hint:** More specific memory text leads to better semantic search results.

---

## 📝 Summary

### **What You Learned:**

1. **The Grounding Problem** - Why agents need memory to resolve references
2. **Working Memory** - Session-scoped conversation history for continuity
3. **Long-term Memory** - Cross-session persistent knowledge for personalization
4. **Memory Integration** - Combining memory with Section 2's RAG system
5. **Complete Context Engineering** - All four context types working together
6. **Production Architecture** - Using Agent Memory Server for scalable memory

### **What You Built:**

- ✅ Working memory demo (multi-turn conversations)
- ✅ Long-term memory demo (persistent knowledge)
- ✅ Complete memory-enhanced RAG system
- ✅ Integration of all four context types

### **Key Functions:**

- `memory_enhanced_rag_query()` - Complete memory + RAG pipeline
- `working_memory_demo()` - Demonstrates conversation continuity
- `longterm_memory_demo()` - Demonstrates persistent knowledge
- `complete_demo()` - End-to-end multi-turn conversation

### **Architecture Pattern:**

```
User Query
    ↓
Load Working Memory (conversation history)
    ↓
Search Long-term Memory (user facts)
    ↓
RAG Search (relevant courses)
    ↓
Assemble Context (System + User + Conversation + Retrieved)
    ↓
Generate Response
    ↓
Save Working Memory (updated conversation)
```

### **From Section 2 to Section 3:**

**Section 2 (Stateless RAG):**
- ❌ No conversation history
- ❌ Each query independent
- ❌ Can't resolve references
- ✅ Retrieves relevant documents

**Section 3 (Memory-Enhanced RAG):**
- ✅ Conversation history (working memory)
- ✅ Multi-turn conversations
- ✅ Reference resolution
- ✅ Persistent user knowledge (long-term memory)
- ✅ Personalization across sessions

### **Next Steps:**

**Section 4** will add **tools** and **agentic workflows** using **LangGraph**, completing your journey from context engineering fundamentals to production-ready AI agents.

---

## 🎉 Congratulations!

You've successfully built a **memory-enhanced RAG system** that:
- Remembers conversations (working memory)
- Accumulates knowledge (long-term memory)
- Resolves references naturally
- Personalizes responses
- Integrates all four context types

**You're now ready for Section 4: Tools & Agentic Workflows!** 🚀




### 🎯 Memory Lifecycle Best Practices

**1. Trust Automatic Extraction**
- Agent Memory Server automatically extracts important facts
- Don't manually store everything in long-term memory
- Let the system decide what's important

**2. Use Appropriate Memory Types**
- Working memory: Current conversation only
- Long-term memory: Facts that should persist

**3. Monitor Memory Growth**
- Long-term memories accumulate over time
- Implement cleanup for outdated information
- Consider archiving old memories

**4. Plan for Expiration**
- Working memory expires after 24 hours
- Important context must be in long-term memory
- Don't rely on working memory for cross-session data

**5. Test Cross-Session Behavior**
- Verify long-term memories are accessible
- Ensure personalization works after TTL expiration
- Test with realistic time gaps

---

## 🎓 Key Takeaways

### **1. Memory Solves the Grounding Problem**

Without memory, agents can't resolve references:
- ❌ "What are **its** prerequisites?" → Agent doesn't know what "its" refers to
- ✅ With working memory → Agent resolves "its" from conversation history

### **2. Two Types of Memory Serve Different Purposes**

**Working Memory (Session-Scoped):**
- Conversation messages from current session
- Enables reference resolution and conversation continuity
- TTL-based (expires after session ends)

**Long-term Memory (Cross-Session):**
- Persistent facts, preferences, goals
- Enables personalization across sessions
- Searchable via semantic vector search

### **3. Memory Completes the Four Context Types**

From Section 1, we learned about four context types. Memory enables two of them:

1. **System Context** (Static) - ✅ Section 2
2. **User Context** (Dynamic, User-Specific) - ✅ Section 2 + Long-term Memory
3. **Conversation Context** (Dynamic, Session-Specific) - ✨ **Working Memory**
4. **Retrieved Context** (Dynamic, Query-Specific) - ✅ Section 2 RAG

### **4. Memory + RAG = Complete Context Engineering**

The integration pattern:
```
1. Load working memory (conversation history)
2. Search long-term memory (user facts)
3. RAG search (relevant documents)
4. Assemble all context types
5. Generate response
6. Save working memory (updated conversation)
```

This gives us **stateful, personalized, context-aware conversations**.

### **5. Agent Memory Server is Production-Ready**

Why use Agent Memory Server instead of simple in-memory storage:
- ✅ **Scalable** - Redis-backed, handles thousands of users
- ✅ **Automatic** - Extracts important facts to long-term storage
- ✅ **Semantic search** - Vector-indexed memory retrieval
- ✅ **Deduplication** - Prevents redundant memories
- ✅ **TTL management** - Automatic expiration of old sessions

### **6. LangChain is Sufficient for Memory + RAG**

We didn't need LangGraph for this section because:
- Simple linear flow (load → search → generate → save)
- No conditional branching or complex state management
- No tool calling required

**LangGraph becomes necessary in Section 4** when we add tools and multi-step workflows.

### **7. Memory Management Best Practices**

**Choose the Right Memory Type:**
- **Semantic** for facts and preferences (most common)
- **Episodic** for time-bound events and timeline
- **Message** for context-rich conversations (use sparingly)

**Understand Memory Lifecycle:**
- **Working memory:** 24-hour TTL, session-scoped
- **Long-term memory:** Indefinite persistence, user-scoped
- **Automatic extraction:** Trust the system to extract important facts

**Benefits of Proper Memory Management:**
- ✅ **Natural conversations** - Users don't repeat themselves
- ✅ **Cross-session personalization** - Knowledge persists over time
- ✅ **Efficient storage** - Automatic deduplication prevents bloat
- ✅ **Semantic search** - Find relevant memories without exact keywords
- ✅ **Scalable** - Redis-backed, production-ready architecture

**Key Principle:** Memory transforms stateless RAG into stateful, personalized, context-aware conversations.

---

## 💪 Practice Exercises

### **Exercise 1: Cross-Session Personalization**

Modify the `memory_enhanced_rag_query` function to:
1. Store user preferences in long-term memory when mentioned
2. Use those preferences in future sessions
3. Test with two different sessions for the same student

**Hint:** Look for phrases like "I prefer...", "I like...", "I want..." and store them as semantic memories.

### **Exercise 2: Memory-Aware Filtering**

Enhance the RAG search to use long-term memories as filters:
1. Search long-term memory for preferences (format, difficulty, schedule)
2. Apply those preferences as filters to `course_manager.search_courses()`
3. Compare results with and without memory-aware filtering

**Hint:** Use the `filters` parameter in `course_manager.search_courses()`.

### **Exercise 3: Conversation Summarization**

Implement a function that summarizes long conversations:
1. When working memory exceeds 10 messages, summarize the conversation
2. Store the summary in long-term memory
3. Clear old messages from working memory (keep only recent 4)
4. Test that reference resolution still works with summarized history

**Hint:** Use the LLM to generate summaries, then store as semantic memories.

### **Exercise 4: Multi-User Memory Management**

Create a simple CLI that:
1. Supports multiple students (different user IDs)
2. Maintains separate working memory per session
3. Maintains separate long-term memory per user
4. Demonstrates cross-session continuity for each user

**Hint:** Use different `session_id` and `user_id` for each student.

### **Exercise 5: Memory Search Quality**

Experiment with long-term memory search:
1. Store 20+ diverse memories for a student
2. Try different search queries
3. Analyze which memories are retrieved
4. Adjust memory text to improve search relevance

**Hint:** More specific memory text leads to better semantic search results.

---

## 📝 Summary

### **What You Learned:**

1. **The Grounding Problem** - Why agents need memory to resolve references
2. **Working Memory** - Session-scoped conversation history for continuity
3. **Long-term Memory** - Cross-session persistent knowledge for personalization
4. **Memory Integration** - Combining memory with Section 2's RAG system
5. **Complete Context Engineering** - All four context types working together
6. **Production Architecture** - Using Agent Memory Server for scalable memory

### **What You Built:**

- ✅ Working memory demo (multi-turn conversations)
- ✅ Long-term memory demo (persistent knowledge)
- ✅ Complete memory-enhanced RAG system
- ✅ Integration of all four context types

### **Key Functions:**

- `memory_enhanced_rag_query()` - Complete memory + RAG pipeline
- `working_memory_demo()` - Demonstrates conversation continuity
- `longterm_memory_demo()` - Demonstrates persistent knowledge
- `complete_demo()` - End-to-end multi-turn conversation

### **Architecture Pattern:**

```
User Query
    ↓
Load Working Memory (conversation history)
    ↓
Search Long-term Memory (user facts)
    ↓
RAG Search (relevant courses)
    ↓
Assemble Context (System + User + Conversation + Retrieved)
    ↓
Generate Response
    ↓
Save Working Memory (updated conversation)
```

### **From Section 2 to Section 3:**

**Section 2 (Stateless RAG):**
- ❌ No conversation history
- ❌ Each query independent
- ❌ Can't resolve references
- ✅ Retrieves relevant documents

**Section 3 (Memory-Enhanced RAG):**
- ✅ Conversation history (working memory)
- ✅ Multi-turn conversations
- ✅ Reference resolution
- ✅ Persistent user knowledge (long-term memory)
- ✅ Personalization across sessions

### **Next Steps:**

**Section 4** will add **tools** and **agentic workflows** using **LangGraph**, completing your journey from context engineering fundamentals to production-ready AI agents.

---

## 🎉 Congratulations!

You've successfully built a **memory-enhanced RAG system** that:
- Remembers conversations (working memory)
- Accumulates knowledge (long-term memory)
- Resolves references naturally
- Personalizes responses
- Integrates all four context types

**You're now ready for Section 4: Tools & Agentic Workflows!** 🚀


